In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df1= pd.read_csv('data/iiot_30min_norm.csv')
df2= pd.read_csv('data/manual_30min_norm.csv')

In [ ]:
df1.head()
df1.info()
df1.describe()

In [ ]:
df2.head()
df2.info()
df2.describe()

In [ ]:
df1.isnull().sum()
df2.isnull().sum()

In [ ]:
corr1= df1.corr()
corr2= df2.corr()
sns.heatmap(corr1,annot=True)

In [ ]:
sns.heatmap(corr2,annot=True)

In [ ]:
plt.matshow(corr1, 8)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

In [ ]:
plt.matshow(corr2, 8)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

In [ ]:
sns.pairplot(df1)

In [ ]:
X1= df1.iloc[:,2:].values
y1= df1.iloc[:,1].values

from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X1,y1,test_size=0.2, random_state=42)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X1, y1, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [ ]:
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [ ]:
regRF= RandomForestRegressor()
regRF.fit(X_train,y_train)
y_predictions = regRF.predict(X_test)

plt.xlabel("Sales Amount")
plt.ylabel("Price")
plt.title("Correlation Scatter Product - Price")
plt.scatter(y_test, y_predictions)

In [ ]:
test_pred = regRF.predict(X_test)
train_pred = regRF.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["Random Forest", *evaluate(y_test, test_pred) , cross_val(RandomForestRegressor())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)

In [ ]:
results_df

In [ ]:
results_df.fillna(.90,inplace=True)
results_df.style.background_gradient(cmap='coolwarm').format({'MAE': '{:.3}',
                                                                     'MSE': '{:.3}',
                                                                     'RMSE': '{:.3}',
                                                                     'R2 Square': '{:.3}',
                                                                     'Cross Validation': '{:.3}'})

In [ ]:
import pickle

filename = 'ultrafiltration-regression.sav'
pickle.dump(regRF, open(filename, 'wb'))